# 第2章  k-近邻算法

## 本章内容

* k-近邻分类算法
* 从文本中解析和导入数据
* 使用Matplotlib创建扩散图
* 归一化数值

## 2.1 k-近邻算法概述

**k-近邻算法**：（kNN, k-NearestNeighbor）是采用测量不同特征值之前的距离的方法来进行分类。

**工作原理**：

1. 存在一个样本数据集，也称作训练样本集，并且样本集中每个数据都存在标签，即已经知道样本集中每一数据与所属分类的对应关系；
2. 输入新的无标签的数据后，将新数据的每个特征与样本集中数据对应的特征进行比较；
    * 计算新数据与样本数据集中每条数据的距离
    * 对求得的所有距离进行排序（从小到大，越小表示越相似）
    * 取前 k （k 一般小于等于 20 ）个样本数据对应的分类标签
3. 求 k 个数据中出现次数最多的分类标签作为新数据的分类。


**开发流程**：

1. 收集数据：任何方法
2. 准备数据：距离计算所需要的数值，最好是结构化的数据格式
3. 分析数据：任何方法
4. 训练算法：此步骤不适用于 k-近邻算法
5. 测试算法：计算错误率
6. 使用算法：首先需要输入样本数据和结构化输出结果，然后运行k-近邻算法判定输入数据分贝属于哪个分类，最后应用对计算出的分类执行后续的处理。

**算法特点**：

* 优点：精度高、对异常值不敏感、无数据输入假定。
* 缺点：计算复杂度高、空间复杂度高
* 使用数据范围：数值型和标称型。


## 2.2 k-近邻算法的使用场景

> 这里通过一个例子进行讲解。

电影可以按照题材进行分类，那么如何区分`动作片`和`爱情片`呢？首先我们知道，`动作片`中肯定打斗的场景居多，而`爱情片`中，亲吻的场景居多。

* `动作片` ：打斗次数更多
* `爱情片` ：亲吻次数更多

那么基于电影的打斗和亲吻出现的次数，使用k-近邻算法构造分类程序，就可以自动划分电影的题材类型了。

假设已经有了如下的数据样本：

![](images/movie_demo.png)

图中问号的部分是我们需要分类的未知电影，既然不知道该电影的类型，只知道其电影中打斗和亲吻出现的次数，那么我们就可以通过某种方法计算出来。首先计算未知电影的特征和其他电影特征的距离，假设结果如下：

![](images/movie_demo_dis.png)


现在根据上面我们得到的样本集中所有电影与未知电影的距离，按照距离递增排序，可以找到 k 个距离最近的电影。

假定 $k=3$，则三个最靠近的电影依次是， He's Not Really into Dudes 、 Beautiful Woman 和 California Man。

k-近邻算法按照距离最近的三部电影的类型，决定未知电影的类型，而这三部电影全是爱情片，因此我们判定未知电影是爱情片。

## 2.3 k-近邻算法的伪代码和实现

**伪代码**

对未知类别属性的数据集中的每个点依次执行以下操作：

1. 计算已知类别数据集中的点与当前点之间的距离；
2. 按照距离递增次序排序；
3. 选取与当前点距离最小的$k$个点；
4. 确定前$k$个点所在类别的出现频率；
5. 返回前$k$个点出现频率最高的类别作为当前点的预测分类。


**Python实现**

In [2]:
from numpy import *
import operator

def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize, 1)) - dataSet
    sqDiffMat = diffMat ** 2
    sqDistances = sqDiffMat.sum(axis = 1)
    distances = sqDistances ** 0.5
    sortDistIndicies = distances.argsort()
    classCount = {}
    for i in range(k):
        voteLabel = labels[sortDistIndicies[i]]
        classCount[voteLabel] = classCount.get(voteLabel, 0) + 1
    sortedClassCount = sorted(classCount.iteritems(), key = operator.itemgetter(1), reverse = True)
    return sortedClassCount[0][0]

上述函数中，共有4个输入参数：
1. inX：用于分类的输入向量，也就是待分类的数据向量
2. dataSet：训练样本集
3. labels：标签向量
4. $k$： 最近邻选择的数目

其中，第6行~第9行代码是**距离计算**部分，第12行~第14行是**选择距离最小的$k$个点**部分，第15行是最终的**排序**部分。

**距离计算**：

这里计算距离使用的是**欧氏距离公式**，计算两个向量点$xA$和$xB$之间的距离：

$$d = \sqrt{{(xA_0 - xB_0)}^2 + {(xA_1 - xB_1)}^2}$$

例如，点$(0, 0)$与$(1, 2)$之间的距离计算为：

$$\sqrt{(1 - 0)^2 + (2 - 0)^2}$$

如果数据存在4个特征值，则点$(1, 0, 0, 1)$与$(7, 6, 9, 4)$之间的距离计算为：

$$\sqrt{(7 - 1)^2 + (6 - 0)^2 + (9 - 0)^2 + (4 - 1)^2}$$